In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [2]:
input_path = 'H:\RediMinds\VCQI'
train = pd.read_csv(input_path+"\VCQI_clean_train_30_day_complications.csv")
test = pd.read_csv(input_path+"\VCQI_clean_test_30_day_complications.csv")

In [3]:
target = '30_day_COMPLICATIONS'
x_train = train.drop(labels=target, axis = 'columns').copy()
y_train = train[target].copy()
x_test = test.drop(labels=target, axis = 'columns').copy()
y_test = test[target].copy() 

In [4]:
print('% pos labels train {:.2f}'.format(y_train.sum()/len(y_train)))
print('% pos labels test {:.2f}'.format(y_test.sum()/len(y_test)))

% pos labels train 0.09
% pos labels test 0.09


### One Hot Encoding Cataegorical Data

In [5]:
# ONE HOT CODE data for training

# Create dummy variables
with open (input_path+'\cat_col_30_day_complications', 'rb') as fp:
    cat_col = pickle.load(fp)


from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')

one_hot_encoded_array = encoder.fit_transform(x_train[cat_col]).toarray()
column_name = encoder.get_feature_names(cat_col)
x_train_OHE =  pd.DataFrame(one_hot_encoded_array, columns= column_name)
x_train = x_train.merge(x_train_OHE, how = 'left', left_index = True, right_index =True) # create dummy variables
x_train = x_train.drop(labels = cat_col, axis = 'columns') # drop original variables

In [6]:
# Create dummy variables
one_hot_encoded_array = encoder.transform(x_test[cat_col]).toarray()
column_name = encoder.get_feature_names(cat_col)
x_test_OHE =  pd.DataFrame(one_hot_encoded_array, columns= column_name)
x_test = x_test.merge(x_test_OHE, how = 'left', left_index = True, right_index =True) # create dummy variables
x_test = x_test.drop(labels = cat_col, axis = 'columns') # drop original variables

In [7]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))
print('% pos labels train {:.2f}'.format(y_train.sum()/len(y_train)))
print('% pos labels test {:.2f}'.format(y_test.sum()/len(y_test)))

Number of records in trainset 1985
Number records in testset 852
% pos labels train 0.09
% pos labels test 0.09


### Defining Pipeline

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [9]:
# Pipeline for logist Classifier
sm = SMOTE(random_state=2)
numeric_features = x_train.select_dtypes('float').columns.tolist()
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),], remainder='passthrough')

### Neural Network Classifier

In [10]:
def nn_model(dropout_rate, neurons, learning_rate):
    from numpy.random import seed
    seed(123)
    from tensorflow.random import set_seed
    set_seed(123)
    tf.keras.backend.clear_session()
   
    # input layer
    input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
    x = keras.layers.Dense(neurons, name = 'Dense_1',activation='relu')(input_layer)
    x = keras.layers.Dropout(dropout_rate, name=  'Dropout_1', seed = 42)(x)
    x = keras.layers.Dense(neurons, name = 'Dense_2',activation='relu')(x)
    x = keras.layers.Dropout(dropout_rate, name=  'Dropout_2', seed = 42)(x)
    main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

    model = keras.Model(inputs= input_layer, outputs=main_output)

    # compiling the model
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                           tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
                  )

    # Keras callback. The patience parameter is the amount of epochs to check for improvement
    
    return model

In [11]:
np.random.seed(1)
tf.random.set_seed(1)
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=nn_model, verbose=0)
# grid search epochs, batch size and optimizer
parameter_dist = {'classifier__dropout_rate':[0.2,0.3, 0.4, 0.5],
                  'classifier__epochs':[20],
                  'classifier__neurons':[128],
                  'classifier__learning_rate': [0.01, 0.005, 0.001, 0.0005, 0.0001],
                 }

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('SMOTE', sm),
                      ('classifier', model)])
model = GridSearchCV(clf,parameter_dist,n_jobs= 1,scoring= 'roc_auc', cv = 10, verbose=1)

In [12]:
#early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',restore_best_weights=True)
model.fit(x_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 20 candidates, totalling 200 fits


W0919 15:49:42.179293 20176 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\python\keras\metrics.py:1792: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 20.0min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('scaler',
                                                                                          StandardScaler(copy=True,
                                                                                                         with_mean=True,
                

In [13]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_label'])
results_nn['pred_prob'] =  pd.DataFrame(model.predict_proba(x_test))[1]
results_nn['true_label'] = np.array(y_test)

In [14]:
results_nn

pred_label  pred_prob  true_label
0             0   0.417581           0
1             1   0.806132           0
2             0   0.002605           0
3             1   0.579471           0
4             0   0.076634           0
5             0   0.110386           0
6             0   0.014321           0
7             0   0.499204           0
8             0   0.168309           0
9             0   0.061692           0
10            0   0.136555           0
11            1   0.763680           1
12            1   0.779710           1
13            0   0.006271           0
14            0   0.005832           0
15            1   0.565710           0
16            0   0.340663           0
17            1   0.580295           0
18            0   0.003641           0
19            0   0.001306           0
20            0   0.030480           0
21            0   0.395392           0
22            0   0.003180           0
23            1   0.808160           0
24            0   0.003297           0
25            1   0.803102           1
26            0   0.000992           0
27            0   0.001277           0
28            0   0.283573           0
29            0   0.000414           0
..          ...        ...         ...
822           1   0.812458           0
823           0   0.011821           0
824           0   0.486920           0
825           1   0.707730           1
826           0   0.066896           0
827           1   0.643917           0
828           0   0.003663           0
829           0   0.004389           0
830           0   0.093874           0
831           0   0.125386           1
832           0   0.250006           0
833           0   0.483622           0
834           1   0.770207           0
835           0   0.007874           0
836           0   0.126085           0
837           0   0.089934           0
838           0   0.215214           0
839           0   0.001107           0
840           0   0.399062           0
841           1   0.714795           0
842           0   0.002932           0
843           1   0.749611           0
844           0   0.058509           0
845           0   0.412219           1
846           1   0.584743           0
847           0   0.269978           0
848           0   0.028478           0
849           0   0.004148           0
850           1   0.643644           0
851           0   0.015368           0

[852 rows x 3 columns]

In [15]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n PR-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.6591369920402178

 Confusion Matrix : 
[[609 166]
 [ 36  41]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.94      0.79      0.86       775
           1       0.20      0.53      0.29        77

    accuracy                           0.76       852
   macro avg       0.57      0.66      0.57       852
weighted avg       0.88      0.76      0.81       852


 AUC-ROC: 
0.7910012568077084

 PR-ROC: 
0.30596301984762003


In [16]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [17]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [18]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

roc_auc_nn     pr_auc_nn
count  10000.000000  10000.000000
mean       0.790812      0.308963
std        0.023498      0.051576
min        0.685292      0.130071
2.5%       0.742904      0.211851
50%        0.791303      0.308436
97.5%      0.835203      0.410124
max        0.880199      0.509808

### Exporting the Model and Pipeline

In [19]:
output_path = 'output/models/'

In [20]:
import os
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [21]:
from joblib import dump
# Save the Keras model first:
model.best_estimator_.named_steps['classifier'].model.save(output_path+'SMOTE_nn_model.h5')

# Finally, save the pipeline:
dump(model.best_estimator_.named_steps['SMOTE'], output_path+'nn_SMOTE_pipeline.pkl')

# Finally, save the pipeline:
dump(model.best_estimator_.named_steps['preprocessor'], output_path+'nn_preprocessor_pipeline.pkl')

['output/models/nn_preprocessor_pipeline.pkl']